<h1>Kryptosystem McEliece oparty o kody Hamminga</h1>

In [ ]:

#Student musi uzupełnić poniższe danne, w przypadku braku/błedu w nr albuma praca studenta nie zostanie zaliczona.
#Prosimy uzupełniać kod wyłącznie w wyznaczonych komórkach z komentarzem Implement me.
NAME=""
NRALBUM=""


<h1>Kod Hamminga</h1>

Dla szyfrowania kryptosystemem McEliece wiadomośći na początku musimy wygenerować klucze publiczne i prywatne (Haminga lub Goppa).

Kluczem prywatnym w systeme będzie zbiór macierzy (S, G, P), gdzie S - odwracalna macierz, G - macierz Hamminga, P - macierz permutacji.
Kluczem publicznym będzie macierz <i>$G'$</i> równa ilorazu tych trzech macierzy. $G' = S * G * P$

Macierz (Kod) Hamminga wykrywa i koryguje błędy polegające na przekłamaniu jednego bitu (ang.) single error correction. Dla niezawodnej transmisji wymagane jest, aby odległość Hamminga między słowami transmitowanym i odbieranym, wynosiła 0 lub 1. Kod ten może wykrywać (ale już nie korygować) błędy podwójne (dwa jednocześnie przekłamane bity), ale nie zawsze.



### Zadanie dla studentów
1. Uzupełń kod algorytmu Hamminga do generacji macierzy G i macierzy kontroli parzystości.
2. Uzupełń kod generowania odwracalnej macierzy S.
3. Uzupełń kod generowania macierzy permutacji P.

In [70]:
import numpy as np
import math

class hamming_keygen:
    "Generates a public key for a message of size 2^kgen-1. Returns G^, the public key, as well as S G and P (Private Key Components)"
    def __init__(self, kgen):
        self.kgen = kgen
        self.k = 2**self.kgen - self.kgen -1
        self.n = 2**self.kgen -1
        self.G = self.genHammingMatrix()
        self.S = self.genInvertibleMatrix()
        self.P = self.genPermuteMatrix()
        self.G_prim = np.matmul(np.matmul(self.S, self.G), self.P) % 2

    def genHammingMatrix(self):
        """Generates Hamming Matrix given k and n"""
        identity = np.identity(self.kgen)
        identityk = np.identity(self.k)
        left = np.zeros((self.kgen, 2**self.kgen - 1 - self.kgen)).T
        rowcount = 0
        for i in range(2**self.kgen):
            if i + 1 != 1:
                if (i + 1) & i != 0:
                    binarystring = np.binary_repr(i+1)
                    column = np.zeros((len(binarystring), 1))
                    for i in range(len(binarystring)):
                        column[-i - 1] = binarystring[i]
                    column = np.pad(column, (0, self.kgen - len(binarystring)), 'constant')
                    left[rowcount] = column.T[0]
                    rowcount += 1
        left = left.T
        self.paritycheck = np.block([left, identity])
        self.generator = np.block([identityk, np.transpose(left)])
        return self.generator
    def genInvertibleMatrix(self):
        "Generates S ,an invertible matrix of size K*l"
        #Implement me
        S = np.random.randint(0,2,(self.k, self.k), dtype=np.uint)
        while np.linalg.det(S) == 0:
            S = np.random.randint(0,2,(self.k, self.k), dtype=np.uint)
        return S       
        

    def genPermuteMatrix(self):
        "Generates P, a random perumutation of the identity matrix"
        #Implement me
        P = np.identity(self.n, dtype=np.uint)
        return P[np.random.permutation(self.n)]
        # P = ...
        #return P


<h1>Szyfrowanie</h1>

Dla szyfrowania kryptosystemem McEliece wiadomośći <i>m</i>, zkonwertowanej do wektora bitów, używamy prostego wzora:

$y = m * G' + e$

Gdzie <i>e</i> - wektor błedu, <i>$G'$</i> - macierz Hamminga (nasz klucz publiczny).

### Zadanie samodzielne dla studentów
1. Uzupełń kod do generacji wektora błedu.
2. Uzupełń kod szyfrujący wiadomość.

In [31]:
#Config
CELLNAME="Encoder"
CELLTYPE="code"
#End
class Encoder:
    '"Encodes a given message, m, using public key g_prime, by permuting and then adding errors to the message."'
    def __init__(self, m, g_prime, t=1):
        self.g_prime = g_prime
        self.message = m
        (k, n) = g_prime.shape
        self.k = k
        self.n = n
        self.t = t
        self.z = self.generate_errors()
        self.encoded = self.encode()

    def generate_errors(self):
        """Generates 1 random errors in bitstring of length n"""
        #Implement
        self.z = np.zeros(self.n)
        idx_list = np.random.choice(self.n, self.t, replace=False)
        for idx in idx_list:
            self.z[idx] = 1
        return self.z
        #End

    def encode(self):
        """Encode message by multiplying by G^, and add random error."""
        #Implement
        self.c_prime = np.matmul(self.message, self.g_prime) % 2
        c = (self.c_prime + self.z) % 2
        return c
        #End

    def get_message(self):
        return self.message

    def get_encrypted(self):
        return self.encoded

<h1>Deszyfrowanie</h1>

Dla deszyfrowania wiadomośći <i>m</i> przypominamy wzór:

$y = m * G' + e$

Przemnożymy wiadomość <i>y</i> przez $P^{-1}$, (P - macierz permutacji, iloczyn tylko zmienia porządek wierszów macierzy. Przemnożenie z $P^{-1}$ zwróci nam początkowy porządek $S*G$).

$y * P^{-1} = (m * G' + e) * P^{-1} = m*G'*P^{-1} + e * P^{-1} = m*(S*G*P)*P^{-1} + z = m*(S*G) + z$

Dalej za pomocą macierzy G i macierzy parzystości H naprawiamy bład (jeżeli on jest) $z$.
Następnie wybieramy bity dannych(pierwsze (n - bity pierzastości)) i przemnożamy ich przez $S^{-1}$ z prawej strony. W wyniku dostajemy oryginalną wiadomość $m$.


### Zadanie samodzielne dla studentów
1. Uzupełń kod do deszyfrowania wiadomości.

In [116]:
#Config
CELLNAME="Decoder"
CELLTYPE="code"
#End
class Decoder:
    def __init__(self, c, S, P, H, m):
        self.c = c
        self.S = S
        self.P = P
        self.H = H
        self.m = m
        self.decrypted = self.decrypt()
        self.correct = (self.m == self.decrypted)

    def decrypt(self):
        "Decrypts a given message given the private key (S, G, and P)"
        #Implement
        P_inv = np.linalg.inv(self.P)
        S_inv = np.linalg.inv(self.S)
        print("S_inv\n" + str(S_inv))
        c_prime = np.matmul(self.c, P_inv)
        print("Message * SG + z = " + str(c_prime))
        m_prime = self.error_correction(c_prime)
        print("Message * S = " + str(m_prime))
        decrypted = np.matmul(m_prime, S_inv) % 2
        return decrypted
        #End

    def error_correction(self, c_prime):
        "Finds the error based on the calculated parity matrix, and flips that bit."
        parity = np.matmul(c_prime, np.transpose(self.H)) % 2
        print("Syndrome =" + str(parity))
        parity_bits = np.ma.size(parity, 0)
        parity_total = 0
        #Calculate Syndrome (From binary representation to integer)
        for i in range(parity_bits):
            parity_total += 2**i * parity[i]
        #if no errors, return the message
        print("Parity_total: " + str(parity_total))
        if (int((parity_total - 1)) & int(parity_total)) == 0:
            return c_prime[0:(c_prime.size - parity_bits)]
        #otherwise, flip the bit with an error in it. Note math.ceil(np.log2(parity_total)-1) is to switch from standard form.
        else:
            error_message = c_prime
            error_bit = int(parity_total - math.ceil(np.log2(parity_total)) - 1)
            print("Error bit: " + str(error_bit))
            if error_message[error_bit] == 1:
                error_message[error_bit] = 0
                return error_message[0:(c_prime.size - parity_bits)]
            elif error_message[error_bit] == 0:
                 error_message[error_bit] = 1
                 return error_message[0:(c_prime.size - parity_bits)]
            else:
                print("This code is broken")

    def is_correct(self):
        return self.correct8

In [132]:

key_info = hamming_keygen(3)
g_prime = key_info.G_prim
encoded = Encoder(np.array([1,0,0,0]), g_prime)
message = encoded.get_message()
encrypted = encoded.get_encrypted()
print("S = \n" + str(key_info.S))
print("G = \n" + str(key_info.G))
print("P = \n" + str(key_info.P))
print("Message = " + str(message))
print("G^ = \n" +str(g_prime))
print("Encoded Message = " + str(encoded.c_prime))
print("Error Introduced =" + str(encoded.z))
print("Encrypted Message = " + str(encrypted))


decoded = Decoder(encrypted, key_info.S, key_info.P, key_info.paritycheck, message)
print("Message = " + str(decoded.decrypted))


S = 
[[0 1 1 1]
 [0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]]
G = 
[[1. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1.]]
P = 
[[1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0]]
Message = [1 0 0 0]
G^ = 
[[0. 1. 0. 1. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 1. 0. 0.]
 [0. 1. 1. 0. 1. 1. 0.]]
Encoded Message = [0. 1. 0. 1. 1. 0. 1.]
Error Introduced =[0. 0. 0. 0. 1. 0. 0.]
Encrypted Message = [0. 1. 0. 1. 0. 0. 1.]
S_inv
[[ 0.  0.  1. -1.]
 [ 1. -1.  0. -1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]
Message * SG + z = [0. 1. 1. 0. 0. 0. 1.]
Syndrome =[1. 1. 1.]
Parity_total: 7.0
Error bit: 3
Message * S = [0. 1. 1. 1.]
Message = [1. 0. 0. 0.]


In [80]:

def test_invertable():
    key_info = hamming_keygen(3)
    assert np.linalg.det(key_info.S), 'S matrix is not invertable'
